# Sensor Based Activity Recoginition 
Challenge: cdl1 - Sensor based Activity Recognition  
Team: Lea Bütler, Manjavy Kirupa, Etienne Roulet, Si Ben Tran  

Aufgabe: DL Modell erstellen

Hier in diesem Notebook erstellen wir unsere Deep Learning Modelle.

# Libraries / Packages

In [15]:
import tensorflow as tf
from sklearn.model_selection import KFold
import numpy as np
import logging
from datetime import datetime
import dataclasses
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn import model_selection as ms
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import os
from tensorflow.keras.callbacks import TensorBoard
import sklearn

tf.debugging.set_log_device_placement(False)

# datetime as filename for logging
now = datetime.now()
date_time_string = now.strftime("%Y-%m-%d_%H-%M-%S")

logging.basicConfig(level=logging.INFO, filename = f"{date_time_string}.txt", filemode='a')
print(tf.config.list_physical_devices())

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [16]:
%tensorboard --logdir logs

UsageError: Line magic function `%tensorboard` not found.


In [17]:
from dataclasses import dataclass
# Static Parameters
@dataclass
class Parameters():
    batch_size: int = 128
    epochs: int = 10
    verbosity: str = "auto"
    step_size: int = 374
    number_folds: int = 2
    output_size: int = 6

# Daten einlesen

In [18]:
# read the CSV file into a DataFrame
data = pd.read_csv("../Sensor_Data-Wrangling-und-EDA/Alle_Messungen_trimmed.csv", index_col=0) 
display("Read Data", data.head(5))

# convert the string time column to datetime
epoch = pd.Timestamp('1970-01-01')

data['time'] = pd.to_datetime(data['time'])

data['time'] = (data['time'] - epoch).apply(lambda x: int(x.total_seconds() * 1000))

# drop the columns that are not needed
data = data.drop(columns=["id", "user"])

le = LabelEncoder()
data["class"] = le.fit_transform(data["class"])

data.head(5)

'Read Data'

,time,Accelerometer_x,Accelerometer_y,Accelerometer_z,Gyroscope_x,Gyroscope_y,Gyroscope_z,Magnetometer_x,Magnetometer_y,Magnetometer_z,Orientation_qx,Orientation_qy,Orientation_qz,id,user,class,id_combined
1,2023-02-27 15:02:17.156,0.000,9.722,1.278,0.0,0.0,0.0,0.000000,0.0,0.000000,-0.21775,-0.719579,0.631111,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
2,2023-02-27 15:02:31.574,-0.728,9.779,0.000,0.0,0.0,0.0,0.000000,0.0,0.000000,-0.42065,0.568356,-0.598058,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
3,2023-02-27 15:02:16.685,0.000,0.000,0.000,0.0,0.0,0.0,0.000000,43.5,-13.143750,0.00000,0.000000,0.000000,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
4,2023-02-27 15:02:32.295,0.000,0.000,0.000,0.0,0.0,0.0,-13.012501,0.0,20.587502,0.00000,0.000000,0.000000,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
5,2023-02-27 15:02:33.285,0.000,0.000,0.000,0.0,0.0,0.0,-22.612501,0.0,20.268751,0.00000,0.000000,0.000000,01_SamsungA22-2023-02-27_15-02-03,Ben_Tran,Laufen,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen


,time,Accelerometer_x,Accelerometer_y,Accelerometer_z,Gyroscope_x,Gyroscope_y,Gyroscope_z,Magnetometer_x,Magnetometer_y,Magnetometer_z,Orientation_qx,Orientation_qy,Orientation_qz,class,id_combined
1,1677510137156,0.000,9.722,1.278,0.0,0.0,0.0,0.000000,0.0,0.000000,-0.21775,-0.719579,0.631111,0,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
2,1677510151574,-0.728,9.779,0.000,0.0,0.0,0.0,0.000000,0.0,0.000000,-0.42065,0.568356,-0.598058,0,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
3,1677510136685,0.000,0.000,0.000,0.0,0.0,0.0,0.000000,43.5,-13.143750,0.00000,0.000000,0.000000,0,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
4,1677510152295,0.000,0.000,0.000,0.0,0.0,0.0,-13.012501,0.0,20.587502,0.00000,0.000000,0.000000,0,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen
5,1677510153285,0.000,0.000,0.000,0.0,0.0,0.0,-22.612501,0.0,20.268751,0.00000,0.000000,0.000000,0,01_SamsungA22-2023-02-27_15-02-03Ben_TranLaufen


In [21]:
# Set the window size and step size
window_size = 300
step_size = 100

y = data["class"].values
X = data.drop(columns=["class", "id_combined"]).values

# display y and X
display("y", y)
display("X", X)

# Create a sliding window of X with the specified window and step sizes
X_windows = np.array([X[i:i+window_size,:] for i in range(0,X.shape[0]-window_size+1,step_size)])

# Reshape X_windows to 3D format (samples, timesteps, features)
timesteps = X_windows.shape[1]
n_features = X_windows.shape[2]
X_windows = X_windows.reshape(-1, timesteps, n_features)

# Create the corresponding y labels for the sliding windows
y_windows = np.array([y[i+window_size-1] for i in range(0,X.shape[0]-window_size+1,step_size)])
y_windows = to_categorical(y_windows, num_classes=6)

# Display the X_windows and y_windows 
#display("X_windows", X_windows)
#display("y_windows", y_windows)

# Split the dataset into training and testing sets
train_X, test_X, train_y, test_y = train_test_split(X_windows, y_windows, test_size=0.2, random_state=42)



'y'

array([0, 0, 0, ..., 5, 5, 5])

'X'

array([[ 1.67751014e+12,  0.00000000e+00,  9.72200012e+00, ...,
        -2.17749998e-01, -7.19578981e-01,  6.31111026e-01],
       [ 1.67751015e+12, -7.27999985e-01,  9.77900028e+00, ...,
        -4.20650005e-01,  5.68355978e-01, -5.98057985e-01],
       [ 1.67751014e+12,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       ...,
       [ 1.67854032e+12,  9.94240170e-01,  3.07503464e+00, ...,
         9.59244333e-01, -4.56052347e-03,  2.82537639e-01],
       [ 1.67854032e+12,  4.23653867e-01,  2.43334855e+00, ...,
         9.59244933e-01, -5.84288178e-03,  2.82514528e-01],
       [ 1.67854032e+12,  1.17072136e-01,  1.64349815e+00, ...,
         9.59104863e-01, -7.07405861e-03,  2.82961185e-01]])

In [22]:
# Something like this as first Model
def create_model_1():
    model = tf.keras.Sequential([
        # Add a 1D convolutional layer
        tf.keras.layers.Conv1D(filters=16, kernel_size=4, activation='relu', padding='same', input_shape=(timesteps, n_features)),
        
        tf.keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
        tf.keras.layers.Dropout(0.2),

        # Add a dense output layer
        tf.keras.layers.Dense(6, activation='softmax')  # Change activation function based on the nature of the output
    ])
    # Change the loss function based on the nature of the output
    model.compile(optimizer='adam',   loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])  
    return model

def create_model_2():
    model = tf.keras.Sequential([
        # Add a 1D convolutional layer
        tf.keras.layers.Conv1D(filters=10, kernel_size=4, activation='relu', padding='same', input_shape=(timesteps, n_features)),
        tf.keras.layers.MaxPooling1D(pool_size=2),
        tf.keras.layers.Conv1D(filters=10, kernel_size=4, activation='relu', padding='same'),
        tf.keras.layers.MaxPooling1D(pool_size=2),

        # flatten output 
        tf.keras.layers.Flatten(),

        # add dense layer with relu activation
        tf.keras.layers.Dense(180, activation='relu'),
        tf.keras.layer.Dense(100, activation='relu'),
        # add output layer with softmax activation
        tf.keras.layers.Dense(6, activation='softmax')  # Change activation function based on the nature of the output

    ])
    # Change the loss function based on the nature of the output
    model.compile(optimizer='adam',   loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])
    return model

def create_model_4():
    model = tf.keras.Sequential([
        # Add a 1D convolutional layer
        tf.keras.layers.Conv1D(filters=64, kernel_size=2, activation='relu', padding='same', input_shape=(timesteps, n_features)),
        
        tf.keras.layers.LSTM(100, dropout=0.2, recurrent_dropout=0.2),
        tf.keras.layers.Dropout(0.2),

        # Add a dense output layer
        tf.keras.layers.Dense(6, activation='softmax')  # Change activation function based on the nature of the output
    ])
    model.compile(optimizer='adam',   loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])  # Change the loss function based on the nature of the output
    return model


In [23]:
best_model_history = None  # Keep track of the best model's history
model_histories = []

models = [create_model_4, create_model_2]
best_model = None


for j, model_creator in enumerate(models):
    model = model_creator()
    logging.info(f'Model {j+1}')
    history = model.fit(train_X, train_y, epochs=Parameters.epochs, batch_size=Parameters.batch_size, validation_data=(test_X, test_y), verbose=Parameters.verbosity)
    test_loss, acc, prec, recal = model.evaluate(test_X, test_y, verbose=Parameters.verbosity)
    logging.info(f'Validation accuracy: {acc}')
    
    for epoch in range(Parameters.epochs):
        # Log accuracy after each epoch
        acc_epoch = history.history['val_accuracy'][epoch]
        logging.info(f'Epoch {epoch + 1}, Validation accuracy: {acc_epoch}')
    
    if best_model_history is None or acc > best_model_acc:
        best_model_history = history
        best_model = model  # Store the trained model instance
        best_model_acc = acc

model_histories.append(fold_histories)

Epoch 1/10
 51/147 [=========>....................] - ETA: 5:56 - loss: 1.9297 - accuracy: 0.1867 - precision_5: 0.2304 - recall_5: 0.0149

KeyboardInterrupt: 

In [ ]:
# Evaluate the best model on the test set
test_loss, test_acc, test_prec, test_recall = best_model.evaluate(x_test, y_test)
logging.info(f'Test accuracy {test_acc}')